# Teil 3: Training und Visualisierung

Hier werden Sie ein neuronales Netzwerk auf den Bilddaten trainieren und testen. Stellen Sie Fragen und versuchen Sie, ein tieferes Verständnis von Faltungsnetzen und Pytorch zu entwickeln.

In [ ]:
"""
Dieser Code kopiert und importiert notwendige Dateien in die virtuelle Maschine von Colab.
"""
import sys, os
if 'google.colab' in sys.modules:
  if os.getcwd() == '/content':
    !git clone 'https://github.com/Criscraft/workshop_ki_hautkrebserkennung.git'
    os.chdir('workshop_ki_hautkrebserkennung')

In [ ]:
import sys
from tqdm import tqdm
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
import torchvision.models as models
import torch
import torch.nn as nn
import numpy as np

from models import Network
import utils

# Konstanten
# TODO: Geben Sie das Verzeichnis Datensatzes an
DATA_TRAIN = 'data/train/'
DATA_TEST = 'data/test/'
MAX_EPOCH = 100
BATCH_SIZE = 64
TRAIN_LOG_PATH = 'train_log.dat'
TEST_LOG_PATH = 'test_log.dat'
#TODO: Ändern Sie NO_CUDA auf False, falls Sie Ihre GPU benutzen wollen
NO_CUDA = False
MODEL_FILE_NAME = 'saved_model.pt'

## 1. Vorbereitung des Netzwerks

Erstellen Sie ein Netzwerk und transferieren Sie es auf die CPU oder die GPU. Wählen Sie das Device abhängig vom Variablenwert ```use_cuda```. Instanzieren Sie die Fehlerfunktion ```nn.CrossEntropyLoss()``` und einen Optimierer (etwa ```torch.optim.SGD```). Setzen Sie die Lernrate zunächst auf 0.01 und den Impuls (momentum) auf 0.9. Vergessen Sie nicht, dem Optimierer die Parameter des Modells ```model.parameters()``` zur Verfügung zu stellen. Der Optimierer wird nur diese Parameter modifizieren, um die Fehlerfunktion zu minimieren.

In [ ]:
use_cuda = not NO_CUDA and torch.cuda.is_available()
cuda_args = {'num_workers': 2, 'pin_memory': True} if use_cuda else {}

# TODO: Erstellen Sie das Pytorch Device ('cuda', falls use_cuda, 'cpu' falls nicht)
device = torch.device("cuda") if use_cuda else torch.device("cpu")

# TODO: erstellen Sie das Netzwerk und transferieren Sie dieses auf das Pytorch Device
model = Network()
#model = models.resnet50(pretrained=True)
#model.fc = nn.Linear(512 * 4, 2)

model = model.to(device)

# TODO: erstellen Sie die Fehlerfunktion
criterion = nn.CrossEntropyLoss()

# TODO: erstellen Sie den Optimierer (stochastic gradient descent)
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)

## Aufgaben

- Was bedeutet die Lernrate?
- Was bedeutet Momentum oder Impuls?
- Was bedeutet num_workers?

## 2. Vorbereitung der Daten

Analog zum ersten Praxisteil erstellen wir für die Trainings- und Validierungsdaten jeweils ein Dataset und einen Dataloader. Wir erstellen unterschiedliche Datasets, da wir unterschiedliche Transformationen auf Trainings- und Validierungsbilder anwenden wollen. Fügen Sie die Transformationen ```transforms.Resize```, ```transforms.ToTensor``` und ```transforms.Normalize``` mittels ```transforms.Compose``` zu einer Transformation zusammen, um die Bilder vorzuverarbeiten. Nutzen Sie für die Normalisierung den Mittelwert und die Standardabweichung, die Sie im ersten Praxisteil bestimmt haben. 

In [ ]:
# Erzeugen Sie eine Transformation, die das Eingabebild erst in einen Tensor umformt und anschließend den Tensor normalisiert.
norm_mean = (0.7811081, 0.53910863, 0.56149995) # skin cancer dataset mean color
norm_std = (0.09854942, 0.08949749, 0.09823854) # skin cancer dataset std 
#norm_mean = (0.485, 0.456, 0.406) # ImageNet mean color
#norm_std = (0.229, 0.224, 0.225) # ImageNet std 

transformations = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=norm_mean, std=norm_std),
    ])

# Erstellen Sie jeweils ein dataset für Training und Validierung
trainset = ImageFolder(DATA_TRAIN, transform=transformations)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)

testset = ImageFolder(DATA_TEST, transform=transformations)
test_loader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE)

## Aufgaben

- Haben Sie die Normalisierung der Daten verstanden?
- Was passiert, wenn Sie die Trainingsdaten nicht shuffeln?
- Prüfen Sie die Größe der beiden Dataloader. Wie viele Minibatches haben sie und wie vielen Bildern entspricht dies?

## 3. Training und Validierung

Nun haben Sie es bis zum Herzstück des Codes geschafft - herzlichen Glückwunsch soweit! Jetzt werden Sie die Trainingsschleife schreiben, in der die Parameter des Netzwerks iterativ verbessert werden. Lassen Sie sich Zeit, den Ablauf genau zu verstehen. Ihr Code sollte schließlich:

1. Mithilfe der Funktionen ```utils.init_log``` eine Logdatei für die Trainingsergebnisse erzeugen - einmal für die Trainingsdaten und noch einmal für die Validierungsdaten
2. In einer for-Schleife über die Epochen iterieren. Nutzen Sie die Funktion tqdm, um einen Ladebalken darzustellen.
3. In einer Schleife über die Minibatches der Trainingsdaten iterieren, mit den Minibatches einen Forward Pass durchführen, den Fehler berechnen und die Parameter des Netzwerkes bei einem Optimierungsschritt anpassen
4. Den aktuellen Fehler und die Genauigkeit auf den Trainingsdaten in die Logdatei schreiben mit ```utils.write_log```.
5. In einer Schleife über die Minibatches der Validierungsdaten iterieren, mit den Minibatches einen Forward Pass durchführen und den Fehler berechnen.
6. Den aktuellen Fehler und die Genauigkeit auf den Validierungsdaten in die Logdatei schreiben.
7. Wenn MAX_EPOCH erreicht ist und das Training beendet ist, das Netzwerk auf der Platte abspeichern. Wir werden Ihr trainiertes Netzwerk morgen noch einmal benötigen!

In [ ]:
#TODO: Initialisieren Sie die beiden Logdateien mit utils.init_log für das Training und das Testen
utils.init_log(TRAIN_LOG_PATH)
utils.init_log(TEST_LOG_PATH)

for epoch in tqdm(range(1, MAX_EPOCH + 1)):
    
    ### Training ###
    
    # Leere Listen 'losses' und 'correct' erstellen. Dort werden wir Zwischenergebnisse akkumulieren.
    losses = []
    correct = []
    n_images = 0
    
    # WICHTIG! Das Netzwerk in den Trainingsmodus umschalten!
    model.train()
    
    for data, target in train_loader:
        # Nebenschleife über die Training Minibatches
        
        n_images += len(target)
        
        # Transferieren Sie data und target auf das Pytorch Device
        data = data.to(device)
        target = target.to(device)
        
        # Die gespeicherten Gradienten auf 0 setzen
        optimizer.zero_grad()
        
        # Berechnen Sie die Ausgabe des Netzwerkes
        outputs = model(data)
        
        # Berechnen Sie den Fehler auf dem Batch
        loss_batch = criterion(outputs, target)

        # Bestimmen Sie die vorhergesagen Labels für die Samples im Batch (Tipp: Benutzen Sie argmax)
        pred = outputs.argmax(1)
        
        # Zählen Sie, wie viele Labels korrekt vorausgesagt wurden.
        correct_batch = (target == pred).sum()
        
        # Berechnung der Gradienten
        loss_batch.backward()
        
        # Optimierungsschritt
        optimizer.step()
        
        # Fügen Sie den Listen 'losses' und 'correct' den aktuellen loss und die Anzahl korrekter Vorhersagen hinzu (Tipp: Benutzen Sie append)
        losses.append(loss_batch.item())
        correct.append(correct_batch.item())
        
    # Wandeln Sie die Listen 'losses' und 'correct' in ndarrays um.
    losses = np.array(losses)
    correct = np.array(correct)
    
    # Berechnen Sie den mittleren Fehler und die mittlere Genauigkeit. Achtung: der loss ist bereits auf den Minibatch gemittelt.
    loss = losses.mean()
    accuracy = correct.sum() / n_images
    
    # Nutzen Sie die write_log Funktion in utils, um die Zwischenergebnisse zu speichern.
    utils.write_log(TRAIN_LOG_PATH, epoch, loss, accuracy)
    
    
    ### Validierung ###
    
    # Leere Listen 'losses' und 'correct' erstellen. Dort werden wir Zwischenergebnisse akkumulieren.
    losses = []
    correct = []
    n_images = 0
    
    # WICHTIG! Das Netzwerk in den Testmodus umschalten!
    model.eval()
    
    # WICHTIG! Zur Validierung die Berechnung der Gradienten ausstellen!
    with torch.no_grad():
        for data, target in test_loader:
            # Nebenschleife über die Test Minibatches
            
            n_images += len(target)
            
            # Transferieren Sie data und target auf das Pytorch Device
            data = data.to(device)
            target = target.to(device)
            
            # Berechnen Sie die Ausgabe des Netzwerkes
            outputs = model(data)
            
            # Berechnen Sie den Fehler auf dem Batch
            loss_batch = criterion(outputs, target)
            
            # Bestimmen Sie die vorhergesagen Labels für die Samples im Batch
            pred = outputs.argmax(1)
            
            # Zählen Sie, wie viele Labels korrekt vorausgesagt wurden.
            correct_batch = (target == pred).sum()
            
            # Fügen Sie den Listen 'losses' und 'correct' den aktuellen loss und die Anzahl korrekter Vorhersagen hinzu
            losses.append(loss_batch.item())
            correct.append(correct_batch.item())
        
    # Wandeln Sie die Listen 'losses' und 'correct' in ndarrays um.
    losses = np.array(losses)
    correct = np.array(correct)
    
    # Berechnen Sie den mittleren Fehler und die mittlere Genauigkeit. Achtung: der loss ist bereits auf den Minibatch gemittelt.
    loss = losses.mean()
    accuracy = correct.sum() / n_images
    
    # TODO: Nutzen Sie die write_log Funktion in utils, um die Zwischenergebnisse zu speichern.
    utils.write_log(TEST_LOG_PATH, epoch, loss, accuracy)

# Abspeichern des trainierten Netzwerks
torch.save(model.state_dict(), MODEL_FILE_NAME)
    
    

## Aufgaben

- Stellen Sie sicher, dass Sie die wesentlichen Schritte verstanden haben. Stellen Sie Fragen!
- An welcher stelle werden die Gradienten berechnet?
- Warum müssen wir nach jedem Batch die Gradienten zurücksetzen?
- An welcher Stelle werden die Gewichte des Netzwerkes verändert?

## 4. Jetzt sind Sie dran!

Hier ein paar Ideen, wie Sie Ihren Algorithmus verbessern können:

- Augmentieren Sie Ihre Trainingsdaten: Bildausschnitte, Skalierung, Kontraständerungen, Weißes Rauschen, Spiegelung, ... was hilft Ihnen bei diesem Problem weiter? Das Paket torchvision hat einige nützliche Funktionen zu bieten!
- Ändern Sie Ihre Netzwerkarchitektur: Machen Sie das Netzwerk tiefer, flacher, weiter oder schmaler. Wie wirken sich Ihre Modifikationen auf Überanpassung und Genauigkeit aus?
- Modifizieren Sie die Lernrate. Probieren Sie, die Lernrate nach einiger Zeit zu verringern. Gewinnen Sie dadurch an Genauigkeit?
- Benutzen Sie einen anderen Optimierer wie Adam.
- Nutzen Sie Regularisierung: Experimentieren Sie z.B. mit Momentum oder Weight Decay. 
- Nutzen Sie ein vortrainiertes Netzwerk
- Falls Sie noch nicht genug haben: Trainieren Sie mehrere Netzwerke (nach einander), bündeln Sie sie zu einem Ensemble und nutzen Sie für die Validierung ihre gemittelte Ausgabe. Was müssen Sie bei der Mittelung der Ausgaben beachten?